In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
pd.set_option('display.max_columns',None)


from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly

sns.set(style='whitegrid')
plt.rcParams['figure.figsize']=(15,5)
%config InlineBackend.figure_format = 'png'
%matplotlib inline

Грузим чеки и магазины

In [2]:
#Грузим Магазины 
oos_by_shop = pd.read_csv("./oos_by_shop_2.csv")

In [3]:
quant_goods = pd.read_excel("./quant_goods.xlsx")

In [4]:
df_price = pd.read_csv("./price.csv")

In [84]:
id_tt_n_tt = pd.read_excel("./id_tt_n_tt.xlsx")

In [5]:
quant_goods

,id_tov,Владелец,Наименование,СтатусАктивностиХарактеристики,БазоваяЕдиницаИзмерения,Вес,Квант,СрокГодности,ТипСрокаГодности,сроксуток,РодительРодитель,Родитель,КатегорияАссортимента
0,46,"Масло сливочное 82,5%",Сычевский,Не активная,кг,1.00,4.8,20,Сутки,20.0,Молочная Продукция,Масло,ОсобыеТовары
1,46,"Масло сливочное 82,5%",Сычевский,Не активная,кг,1.00,4.8,20,Сутки,20.0,Молочная Продукция,Масло,ОсобыеТовары
2,46,"Масло сливочное 82,5%",Агрофирма Оптина,Не активная,кг,1.00,3.4,20,Сутки,20.0,Молочная Продукция,Масло,ОсобыеТовары
3,46,"Масло сливочное 82,5%",Великолукский молочный комбинат ЗАО,Не активная,кг,1.00,2.4,20,Сутки,20.0,Молочная Продукция,Масло,ОсобыеТовары
4,46,"Масло сливочное 82,5%",САПК-Молок ООО,Не активная,кг,1.00,NaN,0,NaN,NaN,Молочная Продукция,Масло,ОсобыеТовары
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58712,66292,"Маракуя микс сырье, кафе",Бреев Андрей Борисович ИП,Новая,шт,0.08,8.0,90,Сутки,90.0,Кафе,Сырьё для кафе,ОсобыеТовары
58713,66293,"Манго дайкири микс сырье, кафе",Бреев Андрей Борисович ИП,Новая,шт,0.08,8.0,90,Сутки,90.0,Кафе,Сырьё для кафе,ОсобыеТовары
58714,66294,"Пиво австрийское ""STIEGL GOLDBRAU""",STIEGL GETRANKE & SERVICE GMBH & CO KG,Новая,шт,0.50,24.0,365,Сутки,365.0,Алкоголь,Пиво,ОсобыеТовары
58715,66295,"Торт-мини ""Йогуртовый"" с персиком",ГРИН КЕЙК ООО,Новая,шт,0.30,2.0,5,Сутки,5.0,"Торты, пирожные, десерты",Торты мини,Новинки


In [6]:
oos_by_shop

,Unnamed: 0,id_tt,id_tov,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,name_tov,name_gr
0,0,11290,52,NaN,4.0,NaN,NaN,NaN,NaN,NaN,Сметана 25%,Молочная Продукция
1,1,11357,52,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Сметана 25%,Молочная Продукция
2,2,11783,52,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Сметана 25%,Молочная Продукция
3,3,12860,52,NaN,NaN,NaN,NaN,2.0,NaN,4.0,Сметана 25%,Молочная Продукция
4,4,15986,52,2.5,NaN,NaN,0.5,0.5,NaN,NaN,Сметана 25%,Молочная Продукция
...,...,...,...,...,...,...,...,...,...,...,...,...
4918,4918,15986,64715,NaN,NaN,NaN,2.0,NaN,NaN,NaN,"Смузи ""Черная смородина""",Напитки
4919,4919,15986,65290,NaN,1.0,NaN,NaN,1.0,1.0,NaN,"Курица ""По-индонезийски"" с кускусом и овощами",Кулинария
4920,4920,15986,65951,2.0,NaN,NaN,NaN,NaN,NaN,1.0,"Омлет с брокколи и сыром, кафе",Кафе
4921,4921,15986,66480,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Крышка ПП для стакана с заглушкой пластик, каф...",Прочее


In [7]:
oos_by_shop.fillna(0, inplace = True)

In [8]:
oos_by_shop_2 = oos_by_shop.copy()

In [9]:
oos_by_shop_2 = oos_by_shop_2.round({'Monday':0,'Tuesday':0,'Tuesday':0,'Wednesday':0,'Thursday':0,'Friday':0,'Saturday':0,'Sunday':0})

In [10]:
oos_by_shop_2

,Unnamed: 0,id_tt,id_tov,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,name_tov,name_gr
0,0,11290,52,0.0,4.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция
1,1,11357,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция
2,2,11783,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция
3,3,12860,52,0.0,0.0,0.0,0.0,2.0,0.0,4.0,Сметана 25%,Молочная Продукция
4,4,15986,52,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция
...,...,...,...,...,...,...,...,...,...,...,...,...
4918,4918,15986,64715,0.0,0.0,0.0,2.0,0.0,0.0,0.0,"Смузи ""Черная смородина""",Напитки
4919,4919,15986,65290,0.0,1.0,0.0,0.0,1.0,1.0,0.0,"Курица ""По-индонезийски"" с кускусом и овощами",Кулинария
4920,4920,15986,65951,2.0,0.0,0.0,0.0,0.0,0.0,1.0,"Омлет с брокколи и сыром, кафе",Кафе
4921,4921,15986,66480,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Крышка ПП для стакана с заглушкой пластик, каф...",Прочее


In [11]:
oos_by_shop_2[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']].sum(axis = 1)

0       4.0
1       1.0
2       1.0
3       6.0
4       2.0
       ... 
4918    2.0
4919    3.0
4920    3.0
4921    1.0
4922    1.0
Length: 4923, dtype: float64

In [12]:

oos_by_shop_2['All_Weekdopost'] = oos_by_shop_2[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']].sum(axis = 1)



In [52]:
oos_by_shop_2

,Unnamed: 0,id_tt,id_tov,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,name_tov,name_gr,All_Weekdopost
0,0,11290,52,0.0,4.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция,4.0
1,1,11357,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция,1.0
2,2,11783,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция,1.0
3,3,12860,52,0.0,0.0,0.0,0.0,2.0,0.0,4.0,Сметана 25%,Молочная Продукция,6.0
4,4,15986,52,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4918,4918,15986,64715,0.0,0.0,0.0,2.0,0.0,0.0,0.0,"Смузи ""Черная смородина""",Напитки,2.0
4919,4919,15986,65290,0.0,1.0,0.0,0.0,1.0,1.0,0.0,"Курица ""По-индонезийски"" с кускусом и овощами",Кулинария,3.0
4920,4920,15986,65951,2.0,0.0,0.0,0.0,0.0,0.0,1.0,"Омлет с брокколи и сыром, кафе",Кафе,3.0
4921,4921,15986,66480,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Крышка ПП для стакана с заглушкой пластик, каф...",Прочее,1.0


In [53]:
oos_by_shop_3 = oos_by_shop_2[(oos_by_shop_2['All_Weekdopost'] != 0) & (oos_by_shop_2['name_tov'] != 'Выдача наличных денег') & (oos_by_shop_2['name_gr'] != 'Прочее') & (oos_by_shop_2['name_gr'] != 'Пространство ВкусВилл') & (oos_by_shop_2['name_gr'] != 'Рекламные материалы')]

In [54]:
oos_by_shop_3[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']].sum(axis = 1).sum(axis = 0)

13858.0

In [55]:
oos_by_shop_3

,Unnamed: 0,id_tt,id_tov,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,name_tov,name_gr,All_Weekdopost
0,0,11290,52,0.0,4.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция,4.0
1,1,11357,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция,1.0
2,2,11783,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция,1.0
3,3,12860,52,0.0,0.0,0.0,0.0,2.0,0.0,4.0,Сметана 25%,Молочная Продукция,6.0
4,4,15986,52,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,4910,15986,62096,0.0,1.0,1.0,0.0,0.0,0.0,0.0,"Ролл с огурцом, 155 г",Кулинария,2.0
4916,4916,15986,64406,0.0,0.0,1.0,0.0,1.0,0.0,0.0,Бургер с куриной котлетой,Кулинария,2.0
4918,4918,15986,64715,0.0,0.0,0.0,2.0,0.0,0.0,0.0,"Смузи ""Черная смородина""",Напитки,2.0
4919,4919,15986,65290,0.0,1.0,0.0,0.0,1.0,1.0,0.0,"Курица ""По-индонезийски"" с кускусом и овощами",Кулинария,3.0


In [56]:
oos_by_shop_3.groupby(["name_gr"], as_index=False)["All_Weekdopost"].sum()

,name_gr,All_Weekdopost
0,Алкоголь,43.0
1,Бакалея,24.0
2,Бытовая Химия,4.0
3,Детские товары,6.0
4,Детское питание,5.0
5,Замороженные п/ф,6.0
6,Кафе,396.0
7,Консервация,21.0
8,Кулинария,4193.0
9,Кулинария замороженная,4.0


In [57]:
quant_goods2 = quant_goods[quant_goods['СтатусАктивностиХарактеристики'] == 'Активная']

In [58]:
quant_goods3 = quant_goods2[['id_tov','РодительРодитель','Владелец','сроксуток']].groupby(["id_tov",'РодительРодитель','Владелец'], as_index=False)["сроксуток"].sum()

In [59]:
oos_by_shop_4 = oos_by_shop_3.merge(quant_goods3[['сроксуток','id_tov']], how='left', on='id_tov',)

In [60]:
conditions = [
    (oos_by_shop_4['сроксуток'] <= 2),
    (oos_by_shop_4['сроксуток'] > 2) & (oos_by_shop_4['сроксуток'] <= 9),
    (oos_by_shop_4['сроксуток'] > 9) & (oos_by_shop_4['сроксуток'] <= 15),
    (oos_by_shop_4['сроксуток'] > 15)
]

In [61]:
values = [7, 5, 2, 1]

In [62]:
oos_by_shop_4['tier'] = np.select(conditions, values)

In [63]:
oos_by_shop_4['tier'].replace(to_replace = 0, value = 1, inplace=True)

In [64]:
oos_by_shop_4['Monday'] = oos_by_shop_4['Monday']/oos_by_shop_4['tier']

In [65]:
oos_by_shop_4['Tuesday'] = oos_by_shop_4['Tuesday']/oos_by_shop_4['tier']


In [66]:
oos_by_shop_4['Wednesday'] = oos_by_shop_4['Wednesday']/oos_by_shop_4['tier']

In [67]:
oos_by_shop_4['Thursday'] = oos_by_shop_4['Thursday']/oos_by_shop_4['tier']

In [68]:
oos_by_shop_4['Friday'] = oos_by_shop_4['Friday']/oos_by_shop_4['tier']

In [69]:
oos_by_shop_4['Saturday'] = oos_by_shop_4['Saturday']/oos_by_shop_4['tier']

In [70]:
oos_by_shop_4['Sunday'] = oos_by_shop_4['Sunday']/oos_by_shop_4['tier']

In [71]:
oos_by_shop_4 = oos_by_shop_4.round({'Monday':0,'Tuesday':0,'Tuesday':0,'Wednesday':0,'Thursday':0,'Friday':0,'Saturday':0,'Sunday':0})


In [72]:
oos_by_shop_4

,Unnamed: 0,id_tt,id_tov,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,name_tov,name_gr,All_Weekdopost,сроксуток,tier
0,0,11290,52,0.0,4.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция,4.0,81.0,1
1,1,11357,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция,1.0,81.0,1
2,2,11783,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция,1.0,81.0,1
3,3,12860,52,0.0,0.0,0.0,0.0,2.0,0.0,4.0,Сметана 25%,Молочная Продукция,6.0,81.0,1
4,4,15986,52,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция,2.0,81.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3754,4910,15986,62096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Ролл с огурцом, 155 г",Кулинария,2.0,3.0,5
3755,4916,15986,64406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Бургер с куриной котлетой,Кулинария,2.0,4.0,5
3756,4918,15986,64715,0.0,0.0,0.0,2.0,0.0,0.0,0.0,"Смузи ""Черная смородина""",Напитки,2.0,21.0,1
3757,4919,15986,65290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Курица ""По-индонезийски"" с кускусом и овощами",Кулинария,3.0,4.0,5


In [73]:
oos_by_shop_4[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']].sum(axis = 1)

0       4.0
1       1.0
2       1.0
3       6.0
4       2.0
       ... 
3754    0.0
3755    0.0
3756    2.0
3757    0.0
3758    0.0
Length: 3759, dtype: float64

In [74]:
oos_by_shop_4['All_Weekdopost'] = oos_by_shop_4[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']].sum(axis = 1)


In [75]:
oos_by_shop_4 = oos_by_shop_4.merge(df_price[['Price_retail','id_tov']], how='left', on='id_tov',)

In [76]:
oos_by_shop_4['overal_sum'] = oos_by_shop_4['Price_retail']* oos_by_shop_4['All_Weekdopost']

In [77]:
oos_by_shop_4['overal_sum'].sum()

686341.2212464431

In [78]:
oos_by_shop_4[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']].sum(axis = 1).sum()

6977.0

In [79]:
oos_by_shop_4[["name_gr","overal_sum"]].groupby(["name_gr"], as_index=False)["overal_sum"].sum().round()

,name_gr,overal_sum
0,Алкоголь,3582.0
1,Бакалея,2028.0
2,Бытовая Химия,327.0
3,Детские товары,250.0
4,Детское питание,290.0
5,Замороженные п/ф,1448.0
6,Кафе,5236.0
7,Консервация,1122.0
8,Кулинария,123735.0
9,Кулинария замороженная,710.0


In [80]:
oos_by_shop_4['id_tt'].unique()

array([11290, 11357, 11783, 12860, 15986, 11956, 12414, 12533, 14112,
       15753], dtype=int64)

In [81]:
oos_by_shop_5 = oos_by_shop_4[oos_by_shop_4['All_Weekdopost'] != 0]

In [82]:
oos_by_shop_5[['name_gr','name_tov']].groupby(["name_tov",'name_gr'], as_index=False)["name_tov"].sum()

,name_gr,name_tov
0,Овощи. Фрукты. Грибы. Зелень,Абрикос
1,Овощи. Фрукты. Грибы. Зелень,Абрикос АзербайджанАбрикос АзербайджанАбрикос ...
2,Овощи. Фрукты. Грибы. Зелень,"Абрикос Крым, 1 кг"
3,Овощи. Фрукты. Грибы. Зелень,"Абрикос Медовый, 500 г"
4,Овощи. Фрукты. Грибы. Зелень,Абрикос Молдавия
...,...,...
877,Мясные деликатесы. Колбасы,"Язык говяжий ""Традиционный""Язык говяжий ""Тради..."
878,Мясо. Птица. П/ф охл. Яйца,Яйцо домашнее от кур бесклеточного содержания
879,Мясо. Птица. П/ф охл. Яйца,Яйцо куриное С0Яйцо куриное С0Яйцо куриное С0Я...
880,Мясо. Птица. П/ф охл. Яйца,Яйцо куриное С1Яйцо куриное С1


In [83]:
oos_by_shop_5

,Unnamed: 0,id_tt,id_tov,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,name_tov,name_gr,All_Weekdopost,сроксуток,tier,Price_retail,overal_sum
0,0,11290,52,0.0,4.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция,4.0,81.0,1,131.818182,527.272727
1,1,11357,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция,1.0,81.0,1,131.818182,131.818182
2,2,11783,52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Сметана 25%,Молочная Продукция,1.0,81.0,1,131.818182,131.818182
3,3,12860,52,0.0,0.0,0.0,0.0,2.0,0.0,4.0,Сметана 25%,Молочная Продукция,6.0,81.0,1,131.818182,790.909091
4,4,15986,52,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Сметана 25%,Молочная Продукция,2.0,81.0,1,131.818182,263.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,4894,15986,47063,0.0,0.0,0.0,0.0,0.0,0.0,2.0,Рулет куриный Айс,Кулинария замороженная,2.0,90.0,1,177.500000,355.000000
3748,4895,15986,50153,0.0,0.0,2.0,0.0,0.0,0.0,0.0,"Сорбет ""Облепиха""",Мороженое,2.0,180.0,1,88.333333,176.666667
3749,4899,15986,51178,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Кукис шоколадный,"Сладости, кондитерские изделия",1.0,18.0,1,70.833333,70.833333
3751,4901,15986,53581,0.0,0.0,0.0,0.0,1.0,1.0,0.0,"Печенье с пармезаном, кунжутом и чесноком","Сладости, кондитерские изделия",2.0,30.0,1,133.333333,266.666667


In [100]:
oos_by_shop_6 = oos_by_shop_5.merge(id_tt_n_tt[['id_tt','n','adress','name_TT']], how = 'left', on = 'id_tt')

In [92]:
oos_by_shop_6.columns

Index(['Unnamed: 0', 'id_tt', 'id_tov', 'Friday', 'Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'name_tov', 'name_gr',
       'All_Weekdopost', 'сроксуток', 'tier', 'Price_retail', 'overal_sum',
       'n', 'adress'],
      dtype='object')

In [101]:
oos_by_shop_7 = oos_by_shop_6[['id_tt','n', 'name_TT','adress', 'id_tov', 'name_tov', 'name_gr', 'Monday',  'Tuesday', 'Wednesday','Thursday','Friday','Saturday','Sunday']]

In [103]:
oos_by_shop_7.head(3)

,id_tt,n,name_TT,adress,id_tov,name_tov,name_gr,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,11290,1020,1020М_Истринская10,Москва ул. Истринская д.10 к.1,52,Сметана 25%,Молочная Продукция,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11357,1087,1087М_Грекова22,"Москва ул. Грекова, д. 22",52,Сметана 25%,Молочная Продукция,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,11783,1501,1501М_Пырьева5,"Москва ул. Пырьева, д. 5а",52,Сметана 25%,Молочная Продукция,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [104]:
oos_by_shop_7.to_excel('Megasigma_postavki.xlsx')